### Creating and Persisting an ML Model

In [89]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/student-mat.csv', sep=';')

Summary of the data

In [90]:
df.describe()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,16.696203,2.749367,2.521519,1.448101,2.035443,0.334177,3.944304,3.235443,3.108861,1.481013,2.291139,3.554430,5.708861,10.908861,10.713924,10.415190
std,1.276043,1.094735,1.088201,0.697505,0.839240,0.743651,0.896659,0.998862,1.113278,0.890741,1.287897,1.390303,8.003096,3.319195,3.761505,4.581443
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,3.000000,0.000000,0.000000
25%,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000,8.000000,9.000000,8.000000
50%,17.000000,3.000000,2.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,4.000000,11.000000,11.000000,11.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,8.000000,13.000000,13.000000,14.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000,19.000000,19.000000,20.000000


In [91]:
df.info

<bound method DataFrame.info of     school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0       GP   F   18       U     GT3       A     4     4   at_home   teacher   
1       GP   F   17       U     GT3       T     1     1   at_home     other   
2       GP   F   15       U     LE3       T     1     1   at_home     other   
3       GP   F   15       U     GT3       T     4     2    health  services   
4       GP   F   16       U     GT3       T     3     3     other     other   
..     ...  ..  ...     ...     ...     ...   ...   ...       ...       ...   
390     MS   M   20       U     LE3       A     2     2  services  services   
391     MS   M   17       U     LE3       T     3     1  services  services   
392     MS   M   21       R     GT3       T     1     1     other     other   
393     MS   M   18       R     LE3       T     3     2  services     other   
394     MS   M   19       U     LE3       T     1     1     other   at_home   

     ... famrel fre

Create a subset of features as an example.

The goal is to predict the quality of the student. We will build a predictor based on the final grade (G3).
Becasue we are trying to find quality students. In this model we define a quality student as one who achieves a final grade of 15 or higher. 

In [92]:
df['qual_student'] = np.where(df['G3']>=15, 1, 0)
df.replace({'no': 0, 'yes': 1}, inplace=True)

In [93]:
df.describe()

,age,Medu,Fedu,traveltime,studytime,failures,schoolsup,famsup,paid,activities,...,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,qual_student
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,...,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,16.696203,2.749367,2.521519,1.448101,2.035443,0.334177,0.129114,0.612658,0.458228,0.508861,...,3.235443,3.108861,1.481013,2.291139,3.554430,5.708861,10.908861,10.713924,10.415190,0.184810
std,1.276043,1.094735,1.088201,0.697505,0.839240,0.743651,0.335751,0.487761,0.498884,0.500555,...,0.998862,1.113278,0.890741,1.287897,1.390303,8.003096,3.319195,3.761505,4.581443,0.388636
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,3.000000,0.000000,0.000000,0.000000
25%,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000,8.000000,9.000000,8.000000,0.000000
50%,17.000000,3.000000,2.000000,1.000000,2.000000,0.000000,0.000000,1.000000,0.000000,1.000000,...,3.000000,3.000000,1.000000,2.000000,4.000000,4.000000,11.000000,11.000000,11.000000,0.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,0.000000,1.000000,1.000000,1.000000,...,4.000000,4.000000,2.000000,3.000000,5.000000,8.000000,13.000000,13.000000,14.000000,0.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,1.000000,1.000000,1.000000,1.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000,19.000000,19.000000,20.000000,1.000000


Drop the G3 score

In [94]:
include1 = ['health', 'absences', 'freetime','studytime','failures','schoolsup','famsup','famrel','Dalc','Walc','qual_student']
include2 = ['health', 'absences', 'age', 'qual_student']
df1 = df.drop(columns=df.columns.difference(include1)) 
df2 = df.drop(columns=df.columns.difference(include2)) 
print(df1.describe())


        studytime    failures   schoolsup      famsup      famrel    freetime  \
count  395.000000  395.000000  395.000000  395.000000  395.000000  395.000000   
mean     2.035443    0.334177    0.129114    0.612658    3.944304    3.235443   
std      0.839240    0.743651    0.335751    0.487761    0.896659    0.998862   
min      1.000000    0.000000    0.000000    0.000000    1.000000    1.000000   
25%      1.000000    0.000000    0.000000    0.000000    4.000000    3.000000   
50%      2.000000    0.000000    0.000000    1.000000    4.000000    3.000000   
75%      2.000000    0.000000    0.000000    1.000000    5.000000    4.000000   
max      4.000000    3.000000    1.000000    1.000000    5.000000    5.000000   

             Dalc        Walc      health    absences  qual_student  
count  395.000000  395.000000  395.000000  395.000000    395.000000  
mean     1.481013    2.291139    3.554430    5.708861      0.184810  
std      0.890741    1.287897    1.390303    8.003096      0

Import scikit-learn and build a random forest classifer

In [109]:
from sklearn.ensemble import RandomForestClassifier as rf
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

dependent_variable = 'qual_student'
x1 = df1.drop(dependent_variable, 1)
y1 = df1[dependent_variable]
x2 = df2.drop(dependent_variable, 1)
y2 = df2[dependent_variable]

xTrain, xTest, yTrain, yTest = train_test_split(x1, y1, test_size=0.2)
xTrain2, xTest2, yTrain2, yTest2 = train_test_split(x2, y2, test_size=0.2)

#baseline model
baseclf = rf(n_estimators=1000)
baseclf.fit(xTrain2, yTrain2)

#our model
ourclf = rf(n_estimators=1000)
ourclf.fit(xTrain, yTrain)

/Users/lizzie/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  import sys
/Users/lizzie/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  if __name__ == '__main__':


RandomForestClassifier(n_estimators=1000)

In [110]:
print('Base Model Report')
pred = baseclf.predict(xTrain2)
print('#Train Set Accuracy: ' + str(accuracy_score(yTrain2, pred)))
pred = baseclf.predict(xTest2)
print('#Test Set Accuracy: ' + str(accuracy_score(yTest2, pred)))
print('Our Model Report')
pred = ourclf.predict(xTrain)
print('#Train Set Accuracy: ' + str(accuracy_score(yTrain, pred)))
pred = ourclf.predict(xTest)
print('#Test Set Accuracy: ' + str(accuracy_score(yTest, pred)))

Base Model Report
#Train Set Accuracy: 0.879746835443038
#Test Set Accuracy: 0.7341772151898734
Our Model Report
#Train Set Accuracy: 0.9936708860759493
#Test Set Accuracy: 0.8227848101265823


In [111]:
import joblib
# modify the file path to where you want to save the model
joblib.dump(ourclf, 'dockerfile/apps/model.pkl')

['dockerfile/apps/model.pkl']